<a href="https://colab.research.google.com/github/yuji-sgs/m1-reserch/blob/main/MOPSO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# はじめに

## 本ノートブックの目的
[前回作成したノートブック](https://github.com/yuji-sgs/m1-reserch/blob/main/Inverse_Analysis.ipynb)では深層学習モデルの1出力に対する逆解析にしか適応できなかったが、複数出力に対する逆解析を実現するために、KerasのFunctional APIを用いて複数出力の深層学習モデルを作成し、その学習済みモデルを適用して多目的粒子群最適化（MOPSO）アルゴリズムを構築していく。

## 処理フロー
1. 使用するライブラリをインポート
2. データの読み込み・確認
3. データ分割
4. KerasのFunctional APIで学習
5. 学習後の精度評価
6. MOPSO実装

## 1. 使用するライブラリをインポート

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import griddata

import io
from google.colab import files

import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, LearningRateScheduler
from keras.models import Model
from keras.layers import Input, Dense

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score

## 2. データの読み込み・確認
ローカルにあるファイルからGoogle Colaboratoryにアップロード

In [2]:
# Google Colaboratoryでファイルをアップロード
uploaded = files.upload()

Saving data_Multiple.csv to data_Multiple.csv


In [3]:
# データフレームとして、変数dfに4列×10000行のデータを格納（C2：材料パラメータ, s：構造パラメータ, w11_max：遮音パラメータ1,　BandGap1：遮音パラメータ2）
df = pd.read_csv(io.BytesIO(uploaded['data_Multiple.csv']))
df

,C2,s,w11_max,BandGap1
0,0.15,0.01,250.00,6.689500e+00
1,0.30,0.01,353.55,5.649800e+00
2,0.45,0.01,433.01,4.264400e+00
3,0.60,0.01,500.00,2.945800e+00
4,0.75,0.01,559.02,1.748300e+00
...,...,...,...,...
9995,14.40,1.00,645.50,4.090000e-12
9996,14.55,1.00,645.50,7.840000e-12
9997,14.70,1.00,645.50,2.270000e-13
9998,14.85,1.00,645.50,4.550000e-12


## 3. データ分割
学習用データとテスト用データに分割

In [4]:
# データの分割
(train, test) = train_test_split(df, test_size=0.2, shuffle=True)

In [5]:
# x_train, x_testではcsvファイルの1, 2列目を， y_train, y_testでは3, 4列目のデータを活用
x_train = train.iloc[:, [0, 1]]
y_train = train.iloc[:, [2, 3]]

x_test = test.iloc[:, [0, 1]]
y_test = test.iloc[:, [2, 3]]

## 4. KerasのFunctional APIで学習
### モデルの概要
- 入力層：2ノード(C2, s)
- 中間層：3層, 各層64ノード
    - アウトプット直前の層にプラスで1層
- 出力層：2ノード（w11_max, BandGap1）

### 活性化関数
- ReLUを採用（勾配消失問題の解消, 処理速度が速い, 多くの場合高い性能を示す）

### 損失関数
- 回帰問題に適したMSE（平均二乗誤差）を採用

### 最適化アルゴリズム
- Adamを採用（Kingma氏らが2014年に提案した高い性能を示すことで知られている | 参考：[Adam: A Method for Stochastic Optimization](https://www.semanticscholar.org/paper/Adam%3A-A-Method-for-Stochastic-Optimization-Kingma-Ba/a6cb366736791bcccc5c8639de5a8f9636bf87e8)）
  - ハイパーパラメータは提案論文の推奨値で決定
    - lr：学習率
    - beta_1：過去の勾配の指数移動平均を計算する際の減衰係数
    - beta_2：過去の勾配の二乗の指数移動平均を計算する際の減衰係数

### その他ハイパーパラメータ
- 学習回数：300回
- バッチサイズ：64

In [6]:
# モデルの構築
# inputの数:C2, s
inputs = Input(shape=(2,))

# 学習回数
epochs = 300
batch_size = 64

# ノード数
node = 64

x1 = Dense(node, activation='relu')(inputs)
x2 = Dense(node, activation='relu')(x1)
x3 = Dense(node, activation='relu')(x2)


x4 = Dense(node, activation='relu')(x3) # アウトプット直前の層
output1 = Dense(1, name='output1')(x4)

x5 = Dense(node, activation='relu')(x3) # アウトプット直前の層
output2 = Dense(1, name='output2')(x5)

model = Model(inputs=inputs, outputs=[output1, output2])

optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
model.compile(
    optimizer=optimizer,
    loss=['mse', 'mse']
)

In [7]:
# 学習オプション
history = model.fit(x_train,
                    {"output1":y_train.iloc[:, [0]], "output2":y_train.iloc[:, [1]]},
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(x_test, {"output1":y_test.iloc[:, [0]], "output2":y_test.iloc[:, [1]]})
)

Epoch 1/300
125/125 [==============================] - 3s 8ms/step - loss: 576034.0625 - output1_loss: 432284.7500 - output2_loss: 143749.2188 - val_loss: 223910.2344 - val_output1_loss: 147560.3438 - val_output2_loss: 76349.9609
Epoch 2/300
125/125 [==============================] - 1s 5ms/step - loss: 208299.5625 - output1_loss: 127994.9609 - output2_loss: 80304.6328 - val_loss: 183188.8125 - val_output1_loss: 103770.1875 - val_output2_loss: 79418.6562
Epoch 3/300
125/125 [==============================] - 0s 3ms/step - loss: 171826.7031 - output1_loss: 91252.0156 - output2_loss: 80574.6328 - val_loss: 145804.9219 - val_output1_loss: 70467.2812 - val_output2_loss: 75337.6328
Epoch 4/300
125/125 [==============================] - 0s 3ms/step - loss: 131355.3906 - output1_loss: 58854.1914 - output2_loss: 72501.2500 - val_loss: 105084.5234 - val_output1_loss: 41464.0664 - val_output2_loss: 63620.4297
Epoch 5/300
125/125 [==============================] - 0s 3ms/step - loss: 99798.5156 -

In [8]:
# 構築したモデルで予測
pred1, pred2 = model.predict(x_test)

63/63 [==============================] - 1s 5ms/step
